**<p style="text-align: center;"><font size= "8" color = "BLACK">WEB SCRAPPING</font></p>**

# About bookpage.com

BookPage is a monthly book review publication, founded in 1988 and located in Nashville, Tennessee. They have a team of editors evaluating and selecting for review the best books in a variety of categories. Authors are required to send a copy of their books to Bookpage.com before their books are published.

# Import libraries

In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
import numpy as np
import pandas as pd
import string
%matplotlib inline
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import plotly.graph_objects as go
import cufflinks as cf
from plotly.offline import iplot
%matplotlib inline
cf.go_offline(connected=True)
from textblob import TextBlob
pd.options.display.float_format = '{:.2f}'.format

# Web scrapping

In this assisgnment, I scrapped the website for the latest 1000 Fiction books (100 pages). 

In the main page, I got the "author" information, "book_cover", "book_title" and links accessing to the "full_review", and the collections of "page_urls". 

Then, in each page url, I accessed to the "full_review" link to get information about the "sub_category", the "published_date" and the "price" of the book.


In [3]:
# next_page_url = "https://bookpage.com/reviews?book_genre=fiction"
# authors= [] 
# images = []
# titles = []
# full_reviews =[]
# pages_urls = ["https://bookpage.com/reviews?book_genre=fiction"]

# for i in range(100):
#     response=requests.get(next_page_url)
#     soup = BeautifulSoup(response.content,"html.parser")
    
#     for a in soup.find_all("p",attrs={"class":"sans bold"}):
#         author = a.get_text().strip()
#         authors.append(author)
        
#     for a in soup.find_all("h4",attrs={"class":"italic"}):
#         title = a.find("a").get_text().strip()
#         titles.append(title)
        
#     for a in soup.find_all("div",attrs={"class":"article-list-left"}):
#         image = a.find("img")["src"]
#         images.append(image)
        
#     for a in soup.find_all("div",attrs={"class":"read-full"}):
#         full_review = "https://bookpage.com" + a.find("a")["href"]
#         full_reviews.append(full_review)


#     next_page = soup.find("div",attrs={"class":"pagination"}).find_all("li")[-2].find("a")["href"]
#     next_page_url = "https://bookpage.com" + next_page
#     pages_urls.append(next_page_url)

In [4]:
# full_contents = []
# prices = []
# published_dates = []
# sub_cats = []
# for url_ in full_reviews:
#     response_ = requests.get(url_)
#     soup_ = BeautifulSoup(response_.content, "html.parser")
#     price = soup_.find_all("div", attrs={"class":"book-data"})[2].find_all("p")[1].text.strip().split("\n")[2]
#     published_date = soup_.find_all("div", attrs={"class":"book-data"})[2].find_all("p")[1].text.strip().split("\n")[-1]
#     full_content=soup_.find("div", attrs={"class":"article-body"}).text.strip().split("\n")
#     sub_cat=soup_.find_all("div",attrs={"class":"book-data"})[-1].find_all("p",attrs={"class":"genre-links"})[-1].find_all("a")[-1].text
#     prices.append(price)
#     published_dates.append(published_date)
#     full_contents.append(full_content)
#     sub_cats.append(sub_cat)

In [5]:
# book_raw_data = pd.DataFrame({"book_title": titles,
#               "author": authors,
#               "published_date": published_dates,
#               "sub_category": sub_cats,
#               "price": prices,
#               "full_review": full_contents,
#               "book_cover": images, 
#              })

# Save and read data

In [6]:
# book_raw_data.to_csv('~/Integrify/assignment/bookpage.csv')

In [7]:
data = pd.read_csv("bookpage.csv",parse_dates=["published_date"],)

# Data analysis

## Data cleaning

### Drop unused columns

In [8]:
data.drop(["Unnamed: 0"],axis=1,inplace=True)

### Quick look at the data

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   book_title      1000 non-null   object
 1   author          1000 non-null   object
 2   published_date  1000 non-null   object
 3   sub_category    1000 non-null   object
 4   price           1000 non-null   object
 5   full_review     1000 non-null   object
 6   book_cover      1000 non-null   object
dtypes: object(7)
memory usage: 54.8+ KB


In [10]:
data.iloc[5:8]

,book_title,author,published_date,sub_category,price,full_review,book_cover
5,★ The Liar's Dictionary,Eley Williams,01/05/2021,Comic Fiction,$26.95,['Two lexicographers employed by the same comp...,https://t1.bookpage.com/books/images/5477caecf...
6,★ A Lie Someone Told You About Yourself,Peter Ho Davies,01/05/2021,Family Drama,$24.00,"['As challenges go, fatherhood can be beautifu...",https://t1.bookpage.com/books/images/027bc6f5a...
7,Better Luck Next Time,Julia Claiborne Johnson,01/05/2021,Historical Fiction,$28.99,"['Back in the 1930s, Dr. Howard Stovall Bennet...",https://t1.bookpage.com/books/images/7fed4451c...


### Format columns

In [11]:
#the book is highly recommended if it´s title starts with *
data["highly_recommended"] = data.book_title.apply(lambda x: not x[0].isalpha())

#remove the special letter from the "highly_recommended" movies
data["book_title"] = data.book_title.apply(lambda x: x.strip("★") if not x[0].isalpha() else x)
data["book_title"] = data.book_title.apply(lambda x: x.strip())

#line index 83 have a year of "20202", it actually was 07/28/2020, instead of 07/08/20202
data["published_date"] = data.published_date.apply(lambda x: x.replace("20202","2020") if x[6:]=="20202" else x)
data["published_date"] = data.published_date.apply(lambda x: x.replace("2106","2016") if x[6:]=="2106" else x)
data["published_date"] = data.published_date.apply(lambda x: x.replace("01/26/2021","01/26/2020") if x=="01/26/2021" else x)
data["published_date"] = pd.to_datetime(data.published_date)

#change "price" to float (there was 27..99,"ISBN" in the original price)
data["price"] = data.price.apply(lambda x: x.strip("$"))
data["price"] = data.price.apply(lambda x: x.replace("..",".") if ".." in x else x)

#clean the full_review column
data["full_review"] = data.full_review.apply(lambda x: x.strip("['").strip("']").replace("', '"," ").replace("\\xa0"," "))

#reset index
data.reset_index(inplace=True)
data.drop(["index"],axis=1,inplace=True)

In [12]:
data.head()

,book_title,author,published_date,sub_category,price,full_review,book_cover,highly_recommended
0,The Push,Ashley Audrain,2021-01-05,Family Drama,26.00,"It’s easy to think of intimate, single-POV nov...",https://t1.bookpage.com/books/images/3c0a5b321...,False
1,The Sea Gate,Jane Johnson,2021-01-05,Family Saga,17.99,Author Jane Johnson’s portfolio is extensive t...,https://t1.bookpage.com/books/images/49b73ef6c...,False
2,A Crooked Tree,Una Mannion,2021-01-05,Coming of Age,27.99,As 15-year-old Libby Gallagher ponders several...,https://t1.bookpage.com/books/images/5f6946511...,False
3,Slash and Burn,"Claudia Hernández, Julia Sanches",2021-01-05,Historical Fiction,17.95,Slash and Burn is an ember of a novel. Origina...,https://t1.bookpage.com/books/images/a9bd55b11...,False
4,The Heiress,Molly Greeley,2021-01-05,Literary Fiction,25.99,Such is Jane Austen’s brilliance that our obse...,https://t1.bookpage.com/books/images/c0e9fff32...,False


### Fill in inappropriate value in the "price" column

In [13]:
#There is "ISBN" in the price
data.price.unique()

array(['26.00', '17.99', '27.99', '17.95', '25.99', '26.95', '24.00',
       '28.99', '27.00', '15.95', '16.00', '16.99', '16.95', '18.00',
       '28.95', '40.00', '28.00', '25.00', '26.99', '25.95', '27.95',
       '17.00', '19.99', '30.00', '22.99', '15.99', '23.95', 'ISBN',
       '23.00', '24.95', '29.99', '24.98', '19.95', '17.50', '14.95',
       '29.95', '14.99', '22.95', '37.50', '29.00', '22.00', '24.99',
       '21.99', '23.99', '12.95', '15.00', '32.50', '36.00', '20.00',
       '13.00', '32.00'], dtype=object)

In [14]:
(data.price == "ISBN").sum()

6

In [15]:
#find the sub_category that have ISBN values
null_subs = data[data.price == "ISBN"].sub_category.unique()
null_subs

array(['Popular Fiction', 'Historical Fiction', 'Audio', 'Coming of Age'],
      dtype=object)

In [16]:
#replace ISBN to zero and all to float64 for further analysis
data.price = data.price.apply(lambda x: "0" if x=="ISBN" else x).astype("float64")
data.price.unique()

array([26.  , 17.99, 27.99, 17.95, 25.99, 26.95, 24.  , 28.99, 27.  ,
       15.95, 16.  , 16.99, 16.95, 18.  , 28.95, 40.  , 28.  , 25.  ,
       26.99, 25.95, 27.95, 17.  , 19.99, 30.  , 22.99, 15.99, 23.95,
        0.  , 23.  , 24.95, 29.99, 24.98, 19.95, 17.5 , 14.95, 29.95,
       14.99, 22.95, 37.5 , 29.  , 22.  , 24.99, 21.99, 23.99, 12.95,
       15.  , 32.5 , 36.  , 20.  , 13.  , 32.  ])

In [17]:
#define a function that returns all outliers in each null_subs and get the mean (after removing outliers), and then return a list
def mean_after_remove_outliers(subs):
    dict_of_mean={}
    for sub in subs:
        sub_data = data[data["sub_category"] == sub]
        Q1 = sub_data.price.quantile(0.25)
        Q3 = sub_data.price.quantile(0.75)
        iqr = Q3 - Q1
        min_= Q1 - 1.5*iqr
        max_= Q3 + 1.5*iqr
        new_sub_data = sub_data.loc[(sub_data.price >= min_) & (sub_data.price <= max_)]
        new_mean = round(new_sub_data.price.mean(),2)
        dict_of_mean[sub] = new_mean
    return dict_of_mean

In [18]:
mean_fill = mean_after_remove_outliers(data.sub_category.unique())
mean_fill

{'Family Drama': 26.65,
 'Family Saga': 26.86,
 'Coming of Age': 26.41,
 'Historical Fiction': 27.21,
 'Literary Fiction': 26.57,
 'Comic Fiction': 26.57,
 'Popular Fiction': 26.69,
 'Dystopian Fiction': 27.58,
 'Fairy Tale': 26.95,
 'Speculative Fiction': 26.59,
 'Short Stories': 26.07,
 'Thriller': 26.74,
 'Anthology': 40.0,
 'Horror': 21.24,
 'Political Fiction': 28.98,
 'Fantasy': 26.7,
 'Satirical Fiction': 26.45,
 'Crime Fiction': 27.0,
 'Gothic Fiction': 26.87,
 'Action & Adventure': 26.48,
 'Science Fiction': 26.61,
 'Science Fiction & Fantasy': 26.98,
 'Historical Fantasy': 27.99,
 'World Fiction': 26.58,
 'Fiction': 25.31,
 'Suspense': 25.99,
 'Audio': 18.48,
 'Debut Fiction': 26.37,
 'Cozy Mystery': 20.98,
 'Western': 23.14,
 'Mystery': 26.12,
 'Inspirational Fiction': 15.66,
 'Mystery & Suspense': 26.0,
 'Historical Mystery': 25.88,
 'Cozies': 25.99,
 'Graphic Novels & Comics': 14.99,
 'Magical Realism': 27.0,
 'Domestic Suspense': 15.99,
 'Romantic Suspense': 26.0,
 'Alter

In [19]:
#fill the missing values
for i in null_subs:
    data.loc[(data.price == 0) & (data.sub_category ==i), "price"] = mean_fill[i]

In [20]:
# data.iloc[[186,261,301,349,361,549]]

## In-depth analysis

### Books published by time

In [21]:
data.published_date.describe(datetime_is_numeric=True).loc[["min","max"]]

min    2016-01-03 00:00:00
max    2021-01-05 00:00:00
Name: published_date, dtype: object

In [22]:
for_bar = pd.pivot_table(index="published_date",values="book_title",aggfunc="count",data=data).resample("ys").sum()
fig = px.bar(for_bar, x=for_bar.index, y='book_title', title='Fiction Books published by years',
            color=for_bar.book_title, color_continuous_scale='Inferno')
fig.add_hline(y= data[data.published_date.dt.year < 2021].book_title.count()/5,
              line_dash="dot", line_color="green", line_width=2)
fig.update_xaxes(title='Years')
fig.update_yaxes(title='Number of Books')
fig.show()

<div style='background-color:navy; color:white; padding:20px'>
    In the last 5 years, Bookpage reviewed approximately 200 Fiction books each year. The largest number of books was reviewed in 2017, yet, in its previous year, 2016, only 141 books were reviewed, reaching the lowest peak.
</div>

In [23]:
for_bar_month = pd.pivot_table(index="published_date",values="book_title",aggfunc="count",data=data).resample("MS").sum()
fig = px.bar(for_bar_month, x=for_bar_month.index, y='book_title', title='Fiction Books published by months',
            color=for_bar_month.book_title, color_continuous_scale='Inferno')
fig.update_xaxes(title='Months')
fig.update_yaxes(title='Number of Books')
fig.show()

<div style='background-color:navy; color:white; padding:20px'>
    <li> 2016 reaching the lowest level in terms of number of books reviewed because there was no books reviewed from Feb to Apr 2016.
    <li>There was no clear pattern which months receiving the highest number of books reviewed. However, in Dec each year, least books were reviewed. It was assumed that in Dec, there are usually long holidays and different services are postponed.
</div>

### Books by sub_categories

In [24]:
#if the sub_cat proportion in the entire dataset less than 0.01 is classifed as "Others"
less_popular_books = pd.DataFrame(data.sub_category.value_counts(normalize=True))
less_popular_books = less_popular_books[less_popular_books["sub_category"] < 0.01]
less_popular_books = less_popular_books.index
data.sub_category = data.sub_category.apply(lambda x: "Others" if x in less_popular_books else x)

In [25]:
def pie_chart(year):
    for_pie = pd.crosstab(index=data.published_date.dt.year,columns=data.sub_category,
                    values=data.book_title,aggfunc="count")
    colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']
    pie_data = pd.DataFrame(for_pie.loc[year])
    pie_data.columns = ["sum_"]
    fig = px.pie(pie_data, values=pie_data.sum_)
    fig.update_traces(textposition='inside',textinfo='percent', textfont_size=10,
                  marker=dict(colors=colors, line=dict(color='#000000', width=1)),
                  hole=.3, hoverinfo="label+percent+name")
    return fig.show()

In [26]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
interact(pie_chart, year = data.published_date.dt.year.unique());

interactive(children=(Dropdown(description='year', options=(2021, 2020, 2019, 2018, 2017, 2016), value=2021), …

In [27]:
# from plotly.subplots import make_subplots
# for_pie=pd.crosstab(index=data.published_date.dt.year,columns=data.sub_category,
#                     values=data.book_title,aggfunc="count")
# labels = for_pie.columns
# colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen']
# specs = [[{'type':'domain'},{'type':'domain'},{'type':'domain'}],
#          [{'type':'domain'},{'type':'domain'},{'type':'domain'}]]
# fig = make_subplots(rows=2, cols=3, specs=specs)
# fig.add_trace(go.Pie(labels=labels, values = for_pie.loc[2016], scalegroup='one',name='2016'),2,3)
# fig.add_trace(go.Pie(labels=labels, values = for_pie.loc[2017], scalegroup='one',name='2017'),2,2)
# fig.add_trace(go.Pie(labels=labels, values = for_pie.loc[2018], scalegroup='one',name='2018'),2,1)
# fig.add_trace(go.Pie(labels=labels, values = for_pie.loc[2019], scalegroup='one',name='2019'),1,3)
# fig.add_trace(go.Pie(labels=labels, values = for_pie.loc[2020], scalegroup='one',name='2020'),1,2)
# fig.add_trace(go.Pie(labels=labels, values = for_pie.loc[2021], scalegroup='one',name='2021'),1,1)

# fig.update_traces(textposition='inside',textinfo='percent', textfont_size=10,
#                   marker=dict(colors=colors, line=dict(color='#000000', width=1)),
#                   hole=.3, hoverinfo="label+percent+name")
# fig.update_layout(title_text= "Book sub_categories by year",
#                  height=1000, width = 1000,legend=dict(orientation="h",
#                                                         yanchor="bottom",
#                                                         y=-0.2,
#                                                         xanchor="right",x=1),
#                   annotations=[dict(text='2021', x=0.12, y=0.8, font_size=1, showarrow=False),
#                                dict(text='2020', x=0.5, y=0.8, font_size=15, showarrow=False),
#                                dict(text='2019', x=0.88, y=0.8, font_size=15, showarrow=False),
#                                dict(text='2018', x=0.12, y=0.2, font_size=15, showarrow=False),
#                                dict(text='2017', x=0.5, y=0.2, font_size=15, showarrow=False),
#                                dict(text='2016', x=0.88, y=0.2, font_size=15, showarrow=False),
#                               ]
#                  )
# fig.show()

<div style='background-color:navy; color:white; padding:20px'>
    <li> Historical fiction and Literary fiction are the two most reviewed sub fiction books on Bookpage over the last 5 years.
    <li> Those two above sub genres maintained their frequency, there was a clear shift. 
    From 2016 to 2018, Debut fiction was either the most or the second most reviewed sub-genres. 
    Yet, from 2019, debut fiction only accounted for 5% of total reviewed books and this sub-genres seemed to disappear in 2020.
</div>

### Books by price

#### Price description

In [28]:
data.price.describe()

count   1000.00
mean      25.68
std        3.30
min       12.95
25%       25.99
50%       26.00
75%       27.00
max       40.00
Name: price, dtype: float64

<div style='background-color:navy; color:white; padding:20px'>
    The cheapest book is 12.95 usd and the most expensive one is 40 usd
</div>

#### Top 10 expensive books

In [29]:
top_10_price = data.iloc[data.price.nlargest(10).index][["book_title","sub_category","price"]]
top_10_price

,book_title,sub_category,price
30,The Neil Gaiman Reader,Others,40.00
438,Lethal White,Mystery,40.00
457,The Labyrinth of the Spirits,Historical Fiction,37.50
678,A Column of Fire,Historical Fiction,36.00
664,Sleeping Beauties,Suspense,32.50
978,Barkskins,Literary Fiction,32.00
97,Utopia Avenue,Historical Fiction,30.00
160,The Mirror & the Light,Historical Fiction,30.00
239,The Institute,Thriller,30.00
293,Deep River,Historical Fiction,30.00


In [30]:
top_10_price.sub_category.value_counts()

Historical Fiction    5
Mystery               1
Thriller              1
Literary Fiction      1
Suspense              1
Others                1
Name: sub_category, dtype: int64

<div style='background-color:navy; color:white; padding:20px'>
    Top 10 most expensive books are over or equal to 30 usd. Of 10 most expensive books, 5 of them are historical fiction.
</div>

#### Price and sub categories

In [31]:
a = pd.DataFrame(data.groupby(["sub_category"]).book_title.count())
b = pd.DataFrame(data.groupby(["sub_category"]).price.agg([np.min,np.max]))
c = pd.DataFrame(pd.Series(mean_fill))
c = c.reset_index()
c.columns = ["sub_category","mean_price"]
c.sub_category = c.sub_category.apply(lambda x: "Others" if x in less_popular_books else x)
c = pd.DataFrame(pd.Series(c.groupby("sub_category").mean_price.mean()))
price_by_sc = pd.concat([a,b,c],axis=1)
price_by_sc.columns = ["total_books","min_price","max_price","avg_price"]
price_by_sc.sort_values(by=["total_books"],ascending=False,inplace=True)
price_others = pd.DataFrame(pd.Series(price_by_sc.loc["Others"])).T
price_by_sc.drop("Others",axis=0,inplace=True)
price_by_sc = pd.concat([price_by_sc,price_others],axis=0)

In [32]:
from plotly.subplots import make_subplots
fig= make_subplots(specs=[[{'secondary_y':True}]])
fig.add_trace(go.Bar(x=price_by_sc.index, y= price_by_sc.total_books,name= "Total books",marker_color=['pink',] * 5,
                    ),secondary_y=False)
fig.add_trace(go.Scatter(mode="markers", x=price_by_sc.index, y= price_by_sc.min_price, name="min_price",marker_line_width=1,
                        marker_color="yellow"),secondary_y=True)
fig.add_trace(go.Scatter(mode="markers", x=price_by_sc.index, y= price_by_sc.max_price, name="max_price",marker_line_width=1),secondary_y=True)
fig.add_trace(go.Scatter(mode="markers", x=price_by_sc.index, y= price_by_sc.avg_price, name="avg_price",
                         marker_symbol="arrow-right", marker_line_color="black", marker_color="red",
                           marker_line_width=1),secondary_y=True)
fig.update_yaxes(title_text="Number of books", secondary_y=False)
fig.update_yaxes(title_text="Price($)", secondary_y=True)
fig.update_layout(title_text= "Book sub_categories and Price",
                 height=700, width = 1000)
fig.show()


<div style='background-color:navy; color:white; padding:20px'>
    <li> The top most 5 frequently reviewed sub_category are Literay, Historical, Debut, Popular Fiction and Comming of Age.
        (Others is not considered for comparison)
    <li> There is also no signifinance among average price accross all sub categories. Yet, historical fiction is the sub_category that has the highest average price (over 27$). 
</div>

### Highly recommended books

#### By sub categories

In [33]:
def recommended_book(col):
    recommend_data = pd.DataFrame(data.pivot_table(index="highly_recommended",columns=col,
                                                   values="book_title",aggfunc="count").iloc[1].dropna().sort_values(ascending=False))
    recommend_data.columns = ["recommended"]
    return recommend_data

In [34]:
labels = data.sub_category.unique()
recommended_by_sc = recommended_book("sub_category")
all_sub_books = pd.DataFrame(data.groupby("sub_category").book_title.count())
colors = ['gold', 'mediumturquoise', 'darkorange', 'lightgreen',]
specs = [[{'type':'domain'},{'type':'domain'}]]
fig = make_subplots(rows=1, cols=2, specs=specs)
fig.add_trace(go.Pie(labels= all_sub_books.index, values = all_sub_books.book_title, hole=.3,
                    pull=[0, 0,0,0.3]),1,1)
fig.add_trace(go.Pie(labels=recommended_by_sc.index, values=recommended_by_sc.recommended, hole=.3,
                    pull=[0, 0, 0.3]
             ),1,2)
fig.update_traces(textposition='inside',textinfo='percent', textfont_size=10,
                  marker=dict(colors=colors, line=dict(color='#000000', width=1)),
                  hole=.3, hoverinfo="label+percent+name")
fig.update_layout(title_text= "Subgenres reviewed and recommended",
                 height=600, width = 1000,
                 legend=dict(orientation="h", yanchor="bottom",y=-0.2,xanchor="right",x=1),
                  annotations=[dict(text='reviewed', x=0.19, y=0.5, font_size=12, showarrow=False),
                               dict(text='recommended', x=0.825, y=0.5, font_size=12, showarrow=False)])



<div style='background-color:navy; color:white; padding:20px'>
    <li> There was no suprise when literary and historical fiction are the two categories that have most books recommended since those two sub categories are the two most common sub_categories over time.
    <li> Yet, Family Drama comes after, occupying the third top subcategory that have books highly recommended. It might indicate either readers favor or Bookpage editor favor in such type of fiction books.
</div>

#### By years

In [35]:
recommended_by_year = recommended_book(data.published_date.dt.year).sort_index()
recommended_by_year

,recommended
published_date,
2017,2.00
2019,69.00
2020,69.00
2021,3.00


<div style='background-color:navy; color:white; padding:20px'>
    Generally, there was approximately 30% of total books reviewed (69/200) were recommended.
    There was almost no books recommended in 2016 and 2017 even Bookpage started reviewing its first fiction book in 1991. There can be several reasons:
    <li> In 2016 and 2017, there was no books of sufficient quality to be highly recommended.
    <li> Bookpage started recommending its reviewed books seriously from 2019.
</div>

### Full_review sentiment analysis

In [36]:
def sentiment_test(col, data=data):
    sentiment_list = []
    for row in data[col]:
        my_valence = TextBlob(row)
        sentiment_list.append(my_valence.sentiment)
    return pd.DataFrame(sentiment_list)

In [37]:
df = pd.concat([data,sentiment_test("full_review")],axis=1)

In [38]:
import plotly.graph_objects as go
fig = go.Figure(data=go.Scatter(x=df['polarity'],
                                y=df['subjectivity'],
                                mode='markers',
                                marker=dict(size=8,
                                            color=df.polarity, 
                                            colorscale='Viridis',
                                            showscale=True,
                                            line_width=0.5),
                                text=data['sub_category']))
fig.add_hline(y=0.6,line_dash="dot", line_color="red", line_width=2)
fig.add_vrect(x0=0, x1=0.4, line_width=0, fillcolor="purple", opacity=0.1, annotation_text = "Positive emotion",
             annotation_position = "inside right")
fig.update_yaxes(title_text="subjectivity")
fig.update_xaxes(title_text="polarity")
fig.update_layout(title_text= "subjectivity and polarity",
                 height=600, width = 1000)
fig.show()

In [39]:
corr_ = df.polarity.corr(df.subjectivity)
corr_

0.1984791673193252

<div style='background-color:navy; color:white; padding:20px'>
    <li> The scatter plot and the corr indicated that there is no significant correlation between polarity and subjectivity.
    <li> A majority of reviews have polarity ranging from 0 to 0.2 and subjectivity from 0.4 to 0.6.
    <li> Most of reviews hold positive emotion, to be specific, hold positive and non_subjective emotion.
    <li> Only 7 reviews contain negative and subjective emotion.
</div>

In [40]:
positive_message = df[df.polarity>0]["book_title"].count()
negative_message = df[df.polarity<0]["book_title"].count()
neutral_message = df[df.polarity==0]["book_title"].count()

print('Neutral emotion in the review:', neutral_message)
print('Positive emotion in the review:', positive_message)
print('Negative emotion in the review:', negative_message)


Neutral emotion in the review: 0
Positive emotion in the review: 905
Negative emotion in the review: 95


# Book Exploration

## All book displays

In [41]:
left_aligned_data = data.style.set_properties(**{'text-align': 'left'}) 

In [42]:
def image_path(path):
    return '<img src ="'+path+'"width ="100">'

from IPython.core.display import HTML
HTML(data[["book_title","published_date","sub_category","book_cover","price","full_review"]].to_html(escape=False,
                                                                                                     formatters=dict(book_cover=image_path)))

,book_title,published_date,sub_category,book_cover,price,full_review
0,The Push,2021-01-05,Family Drama,,26.00,"It’s easy to think of intimate, single-POV novels as somewhat simple narrative exercises, but Ashley Audrain’s gripping debut is proof that this is an illusion. In the hands of the right storyteller, even the most compact novels can be works of great complexity. The Push unfolds through the mind and pen of Blythe, an aspiring writer whose decision to become a mother is weighted against her own difficult childhood. Blythe is determined to be the mother she never had, but her first child, Violet, doesn’t make that easy. Blythe’s husband gets along with their daughter fine, but Blythe can’t help but think that something is off, particularly when their second child gives her the kind of parenting relationship she always wanted. Even then, the feeling that something is not quite right about Violet persists, until it goes so far that Blythe’s entire world is altered in a single shattering moment. The Push is a dazzling exercise in both economy of language and vividness of expression. Audrain’s grasp of Blythe’s inner life—her fears, her hopes, the details that linger in her mind— is so precise and mature that we get lost in this woman’s often troubling world. That feeling propels the novel forward at a blistering pace, but Audrain doesn’t stop there. This is just one woman’s side of the story—a woman who’s a writer, at that—so even as we feel we know Blythe, we can’t help but wonder how much of what she’s telling us is fiction. That this suspicion can coexist with the intimacy of Blythe’s narration is proof of Audrain’s skill as a storyteller and makes the book that much more spellbinding. The Push announces Audrain as a sophisticated, compelling writer, perfect for fans of thrillers and intimate family dramas alike."
1,The Sea Gate,2021-01-05,Family Saga,,17.99,"Author Jane Johnson’s portfolio is extensive to say the least. An editor, novelist and historian, she has edited for Dean Koontz and other big names, and written several novels for children and adults, including the film tie-in books to The Lord of the Rings and Hobbit movies. Her latest novel, The Sea Gate, is a beautiful story of love, loss and pain, bouncing between wartime memories of the 1940s and present day. Set in the gorgeous seascape of Cornwall, England, the novel explores the trials and tribulations of two women: bold and eccentric Olivia Kitto, now in her 90s; and Becky, her 30-something cousin who has learned about Olivia through letters she discovered after her mother’s very recent passing. After an unexpected fall that led to a stint in the hospital, Olivia desperately needs help restoring her beloved home in Cornwall. Becky, needing an escape from a despondent life in London, decides she’ll take on the responsibility that would have otherwise belonged to her late mother. Olivia’s memory is not quite what it used to be, but she remembers enough to know that the renovation of her home will need to include a few repairs that could quite literally unearth some troublesome secrets that she has managed to hide for so long. Becky, being naturally inquisitive, uncovers them anyway and eventually connects the dots that tell a scary tale of murders, Nazis and the courage it takes to protect the ones you truly love at any cost. All of that—plus dives into history, a mind-blowing landscape and even a love story—make The Sea Gate a pleasant read."
2,A Crooked Tree,2021-01-05,Coming of Age,,27.99,"As 15-year-old Libby Gallagher ponders several dark moments in rock ’n’ roll history, she muses, “It all said to me that chaotic and dark forces were spinning around us. One foot wrong, and you’d be pulled into the vortex.” Unfortunately, a multitude of missteps have already affected Libby and her family, and that vortex threatens to loom closer every day in Una Mannion’s taut, richly imagined debut, A Crooked Tree. Living near Valley Forge, Pennsylvania, in the early

## Find books by published year and sub_genres

### First approach

In [43]:
def sunburst_by_year(year):
    year_data = data.loc[data.published_date.dt.year == year]
    fig = px.sunburst(year_data, path=['sub_category', 'book_title'])
    fig.update_layout(title_text= "Book by years",
                 height=700, width = 700)
    fig.show()

In [44]:
interact(sunburst_by_year, year = data.published_date.dt.year.unique())

interactive(children=(Dropdown(description='year', options=(2021, 2020, 2019, 2018, 2017, 2016), value=2021), …

<function __main__.sunburst_by_year(year)>

In [45]:
from PIL import Image
import requests
from io import BytesIO

def display(title):
    display_data = data[data.book_title == title].book_cover.to_list()[0]
    response = requests.get(display_data)
    img = Image.open(BytesIO(response.content))
    print("Author:", data[data.book_title == title].author.to_list()[0])
    print("Published on:", data[data.book_title == title].published_date.to_list()[0])
    print("Sub genre:", data[data.book_title == title].sub_category.to_list()[0])
    print("Price ($):", data[data.book_title == title].price.to_list()[0])
    print(data[data.book_title == title].full_review.to_list()[0])
    return img

In [46]:
interact(display, title=data.sort_values(by=["book_title"]).book_title.unique());

interactive(children=(Dropdown(description='title', options=('10 Minutes 38 Seconds in This Strange World', 'A…

### Second approach

In [47]:
def get_year(year):
    sub_genre = data[data.published_date.dt.year == year].sub_category.unique()
    def get_sub(sub_genre):
        book_list = data[(data.sub_category == sub_genre) & (data.published_date.dt.year == year)].book_title.sort_values().unique()
        def get_book(title):
            display_data = data[(data.book_title == title)].book_cover.to_list()[0]
            response = requests.get(display_data)
            img = Image.open(BytesIO(response.content))
            print("Author:", data[data.book_title == title].author.to_list()[0])
            print("Published on:", data[data.book_title == title].published_date.to_list()[0])
            print("Sub genre:", data[data.book_title == title].sub_category.to_list()[0])
            print("Price ($):", data[data.book_title == title].price.to_list()[0])
            print(data[data.book_title == title].full_review.to_list()[0])

            return img
        return interact(get_book, title=book_list);
    return interact(get_sub,sub_genre= sub_genre);

In [48]:
interact(get_year, year=data.published_date.dt.year.unique());

interactive(children=(Dropdown(description='year', options=(2021, 2020, 2019, 2018, 2017, 2016), value=2021), …

In [49]:
# top_5_sc = price_by_sc.iloc[:5].index
# top_5_price_by_sc = data[data.sub_category.isin(top_5_sc)][["sub_category","price"]]
# y_array = [np.array(top_5_price_by_sc[top_5_price_by_sc.sub_category == i].price) for i in top_5_sc]

# colors = ['rgba(93, 164, 214, 0.5)', 'rgba(255, 144, 14, 0.5)', 'rgba(44, 160, 101, 0.5)',
#           'rgba(255, 65, 54, 0.5)', 'rgba(207, 114, 255, 0.5)']

# fig = go.Figure()

# for x, y, cls in zip(top_5_sc, y_array, colors):
#         fig.add_trace(go.Box(y=y,name=x,boxpoints='all',jitter=0.5,whiskerwidth=0.2,
#             fillcolor=cls,marker_size=2,line_width=1))

# fig.update_layout(
#     title='Price by the top 5 popular sub_category',
#     yaxis=dict(autorange=True,showgrid=True,zeroline=True,dtick=5,gridcolor='rgb(255, 255, 255)',gridwidth=1,zerolinecolor='rgb(255, 255, 255)',zerolinewidth=2,),
#     margin=dict(l=40,r=30,b=80,t=40,),
#     paper_bgcolor='rgb(243, 243, 243)',
#     plot_bgcolor='rgb(243, 243, 243)',
#     showlegend=False)

# fig.show()